In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

# Project Definition

Kasus yang akan digunakan adalah membuka akun deposit untuk nasabah bank. Pada kesempatan kali ini data scientist diminta tolong untuk menganalisa hasil AB-testing suatu perusahaan bank. Apakah jumlah campaign mampu meningkatkan keinginan user untuk membuka akun deposit atau tidak?

Data yang digunakan berasal dari folder bank.zip ﬁle yang digunakan adalah **bank-full.csv**
1. Untuk variabel jumlah campaign bisa dicek untuk kolom **campaign**
2. Untuk variabel apakah akhirnya user membuka akun atau tidak bisa menggunakan **variabel y**

# Pemahaman Konsep A/B Testing

**Nama eksperimen**

Pengaruh Jumlah Campaign terhadap Keputusan Membuka Akun Deposit

**Definisi Hipotesis**
- Hipotesis Nol (H₀): Mean campaign untuk grup "yes" ≠ Mean campaign untuk grup "no". (jumlah campaign tidak berpengaruh terhadap kemungkinan user membuka akun deposit)
- Hipotesis Alternatif (H₁): Mean campaign untuk grup "yes" ≠ Mean campaign untuk grup "no". (jumlah campaign berpengaruh terhadap kemungkinan user membuka akun deposit)

**Siapa participant-nya?**

Semua nasabah (user) yang masuk dalam data bank-full.csv (total 4.521 user).

**Variabel yang diuji**
- Independen (X): Jumlah campaign (kolom campaign)
- Dependen (Y): Keputusan membuka akun (y, bernilai yes atau no)

**Metrics yang digunakan**
- Conversion Rate (jumlah user yang mengatakan “yes”)
- Bisa dibandingkan berdasarkan jumlah campaign menggunakan rata-rata campaign pada dua grup: y = yes vs y = no.

**Sample size dan durasi eksperimen**
- Sample Size: 4.521 data observasi
- Durasi: Tidak disebutkan dalam data, jadi diasumsikan selesai pada saat data dikumpulkan (retrospektif analysis, bukan live A/B Test)

# Pengujian Hipotesis

### A: Penentuan Hipotesis

- **H₀**: Mean campaign untuk grup "yes" = Mean campaign untuk grup "no"
- **H₁**: Mean campaign untuk grup "yes" ≠ Mean campaign untuk grup "no"

### B: Pengujian yang Digunakan

Uji T dua sampel (Independent T-Test) karena:
- Variabel target y terdiri dari dua grup (yes/no)
- Ingin membandingkan rata-rata campaign di dua grup tersebut

### C: Deteksi Asumsi dan Penanganan

Sebelum melakukan uji T dua sampel, kita perlu memastikan beberapa asumsi dasar terpenuhi:

1. **Independensi Observasi**
    - Diasumsikan data diambil secara independen. Karena ini adalah data retrospektif dan observasi individual terhadap nasabah, maka asumsi ini dianggap terpenuhi.
2. **Distribusi Normal** dari masing-masing grup (jumlah campaign untuk user "yes" dan "no")
    - Dapat dicek dengan uji normalitas seperti Shapiro-Wilk atau dengan melihat histogram/QQ-plot.
    - Namun, karena ukuran sampel cukup besar (>30 untuk masing-masing grup), Central Limit Theorem berlaku, sehingga asumsi normalitas dapat dilonggarkan.
3. **Homogenitas varians** (kesamaan varians) antar dua grup
    - Bisa diuji dengan Levene’s Test.
    - Jika hasil Levene menunjukkan varians tidak sama, maka gunakan Welch’s t-test.

### D: Hasil Pengujian Hipotesis

In [11]:
bank = pd.read_csv("datasets/bank-full.csv", sep=";")
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [12]:
# Pisahkan data berdasarkan keputusan membuka akun
campaign_yes = bank[bank['y'] == 'yes']['campaign']
campaign_no = bank[bank['y'] == 'no']['campaign']

print(f"Jumlah Yes: {campaign_yes.shape[0]}")
print(f"Jumlah No: {campaign_no.shape[0]}")

Jumlah Yes: 5289
Jumlah No: 39922


In [17]:
# Hitung statistik deskriptif
mean_yes = campaign_yes.mean()
mean_no = campaign_no.mean()
std_yes = campaign_yes.std()
std_no = campaign_no.std()

print(f"Mean Yes: {mean_yes.round(2)}")
print(f"Mean No: {mean_no.round(2)}")

Mean Yes: 2.14
Mean No: 2.85


In [18]:
# Uji Levene untuk homogenitas varians
levene_test = stats.levene(campaign_yes, campaign_no)

print(f"Levene's test p-value: {levene_test.pvalue}")

Levene's test p-value: 8.220318168029953e-33


In [19]:
# Uji T Dua Sampel (Welch jika varians tidak sama)
t_test = stats.ttest_ind(campaign_yes, campaign_no, equal_var=False)

print(f"\nRata-rata campaign (yes): {mean_yes:.2f}")
print(f"Rata-rata campaign (no): {mean_no:.2f}")
print(f"Statistik t: {t_test.statistic:.4f}")
print(f"p-value: {t_test.pvalue:.6f}")


Rata-rata campaign (yes): 2.14
Rata-rata campaign (no): 2.85
Statistik t: -22.8007
p-value: 0.000000


In [16]:
# Interpretasi
if t_test.pvalue <= 0.05:
    print("Tolak H₀.\nTerdapat perbedaan signifikan antara jumlah campaign pada grup 'yes' dan 'no'.\nJumlah campaign berpengaruh terhadap kemungkinan user membuka akun deposit.")
else:
    print("Gagal menolak H₀.\nTidak terdapat perbedaan signifikan antara jumlah campaign pada grup 'yes' dan 'no'.\nJumlah campaign tidak berpengaruh terhadap kemungkinan user membuka akun deposit.")

Tolak H₀.
Terdapat perbedaan signifikan antara jumlah campaign pada grup 'yes' dan 'no'.
Jumlah campaign berpengaruh terhadap kemungkinan user membuka akun deposit.


## E: Kesimpulan dan Interpretasi

**Kesimpulan**:
- Karena p-value <= 0.05, kita menolak hipotesis nol (H₀) yang menyatakan tidak ada perbedaan jumlah campaign antara grup 'yes' dan 'no'.
- Ini berarti, jumlah campaign berpengaruh secara signifikan terhadap keputusan user membuka akun deposit.
- Namun, hasil ini menunjukkan bahwa rata-rata jumlah campaign untuk grup yang membuka akun (2.27) lebih rendah dibandingkan dengan yang tidak membuka akun (2.86).

**Interpretasi**:
- Meskipun lebih banyak campaign umumnya dianggap dapat meningkatkan konversi, dalam hal ini user yang membuka akun cenderung menerima lebih sedikit campaign.
- Kesimpulan ini bisa menunjukkan bahwa campaign yang terlalu banyak atau agresif mungkin malah berdampak negatif, mengurangi minat atau keinginan user untuk membuka akun deposit.
- Ini mengindikasikan bahwa campaign yang terlalu sering atau agresif mungkin mengganggu atau menurunkan minat user, sehingga berpotensi kontraproduktif.